# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

c:\Users\jackw\.conda\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step4 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step5 结合分词器创建Dataloader

In [14]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

: 

## Step6 创建模型

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 7 创建评估函数

In [6]:
import evaluate
acc_metric  = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")

In [7]:

def eval_metric(eval_predict: tuple):
    
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc


## Step8  创建TrainingArguments


In [8]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=11000,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

c:\Users\jackw\.conda\envs\transformers\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=2,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
evaluation_strategy=epoch,
fp16=

## Step9 创建Trainer

In [9]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)
trainer.train()

  3%|▎         | 10/330 [00:05<01:34,  3.40it/s]

{'loss': 0.6306, 'grad_norm': 1.5908437967300415, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}


  6%|▌         | 20/330 [00:07<01:16,  4.04it/s]

{'loss': 0.5495, 'grad_norm': 4.218531608581543, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}


  9%|▉         | 30/330 [00:10<01:13,  4.07it/s]

...

                                                 
100%|██████████| 330/330 [01:29<00:00,  4.01it/s]

{'eval_loss': 0.24387605488300323, 'eval_accuracy': 0.9009009009009009, 'eval_f1': 0.927699530516432, 'eval_runtime': 1.0339, 'eval_samples_per_second': 751.539, 'eval_steps_per_second': 6.771, 'epoch': 3.0}


100%|██████████| 330/330 [01:29<00:00,  3.67it/s]

{'train_runtime': 89.8259, 'train_samples_per_second': 233.385, 'train_steps_per_second': 3.674, 'train_loss': 0.30736470222473145, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.30736470222473145, metrics={'train_runtime': 89.8259, 'train_samples_per_second': 233.385, 'train_steps_per_second': 3.674, 'total_flos': 351909933963264.0, 'train_loss': 0.30736470222473145, 'epoch': 3.0})

## Step10 模型训练

In [10]:
trainer.train()

  3%|▎         | 10/330 [00:02<01:21,  3.92it/s]

{'loss': 0.2886, 'grad_norm': 4.049503803253174, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}


  6%|▌         | 20/330 [00:05<01:18,  3.94it/s]

{'loss': 0.1943, 'grad_norm': 5.238736152648926, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}


  9%|▉         | 30/330 [00:07<01:16,  3.91it/s]

...

                                                 
100%|██████████| 330/330 [01:27<00:00,  4.03it/s]

{'eval_loss': 0.24040617048740387, 'eval_accuracy': 0.9073359073359073, 'eval_f1': 0.9324577861163227, 'eval_runtime': 1.1149, 'eval_samples_per_second': 696.907, 'eval_steps_per_second': 6.278, 'epoch': 3.0}


100%|██████████| 330/330 [01:27<00:00,  3.75it/s]

{'train_runtime': 87.9217, 'train_samples_per_second': 238.439, 'train_steps_per_second': 3.753, 'train_loss': 0.19635908423048076, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.19635908423048076, metrics={'train_runtime': 87.9217, 'train_samples_per_second': 238.439, 'train_steps_per_second': 3.753, 'total_flos': 351909933963264.0, 'train_loss': 0.19635908423048076, 'epoch': 3.0})

## Step11 模型评估


In [11]:
trainer.evaluate(tokenized_datasets["test"])

100%|██████████| 7/7 [00:00<00:00,  7.50it/s]


{'eval_loss': 0.24040617048740387,
 'eval_accuracy': 0.9073359073359073,
 'eval_f1': 0.9324577861163227,
 'eval_runtime': 1.0931,
 'eval_samples_per_second': 710.809,
 'eval_steps_per_second': 6.404,
 'epoch': 3.0}

## Step12 模型预测


In [12]:
trainer.predict(tokenized_datasets["test"])

100%|██████████| 7/7 [00:00<00:00,  7.78it/s]


PredictionOutput(predictions=array([[-1.9528311 ,  2.7549913 ],
       [-2.52439   ,  3.4644213 ],
       [-2.3588989 ,  3.5158489 ],
       ...,
       [-2.6295562 ,  3.4607577 ],
       [-1.8692572 ,  3.2497716 ],
       [-0.25713792,  0.90523815]], dtype=float32), label_ids=array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1

In [13]:
from transformers import pipeline

id2_label = id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9924901127815247}]